In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data

import random, os, pathlib, time
from tqdm import tqdm
from sklearn import datasets

In [2]:
import nflib
from nflib.flows import SequentialFlow, NormalizingFlow, ActNorm, ActNorm2D, AffineConstantFlow
import nflib.coupling_flows as icf
import nflib.inn_flow as inn
import nflib.res_flow as irf

In [3]:
from torch import distributions
from torch.distributions import MultivariateNormal

In [4]:
device = torch.device("cuda:0")
# device = torch.device("cpu")

## MNIST dataset

In [5]:
import mylibrary.datasets as datasets
# import mylibrary.nnlib as tnn

In [6]:
mnist = datasets.FashionMNIST()
# mnist.download_mnist()
# mnist.save_mnist()
train_data, train_label_, test_data, test_label_ = mnist.load()

train_data = train_data / 255.
test_data = test_data / 255.

# train_label = tnn.Logits.index_to_logit(train_label_)
train_size = len(train_label_)

In [7]:
## converting data to pytorch format
train_data = torch.Tensor(train_data)
test_data = torch.Tensor(test_data)
train_label = torch.LongTensor(train_label_)
test_label = torch.LongTensor(test_label_)

In [8]:
input_size = 784
output_size = 10

In [9]:
class MNIST_Dataset(data.Dataset):
    
    def __init__(self, data, label):
        self.data = data.reshape(-1, 1, 28, 28)
        self.label = label
        
#         self.label = mask.type(torch.float32).reshape(-1,1)
        self._shuffle_data_()
        
    def __len__(self):
        return len(self.data)
    
    def _shuffle_data_(self):
        randidx = random.sample(range(len(self.data)), k=len(self.data))
        self.data = self.data[randidx]
        self.label = self.label[randidx]
    
    def __getitem__(self, idx):
        img, lbl = self.data[idx], self.label[idx]
        return img, lbl

In [10]:
train_dataset = MNIST_Dataset(train_data, train_label)
test_dataset = MNIST_Dataset(test_data, test_label)

In [11]:
class ConnectedClassifier_Linear(nn.Module):
    
    def __init__(self,input_dim, num_sets, output_dim, inv_temp=1):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.num_sets = num_sets
        self.inv_temp = nn.Parameter(torch.ones(1)*inv_temp)
        
        self.linear = nn.Linear(input_dim, num_sets)
#         self.linear.bias.data *= 0
#         self.linear.weight.data *= 0.1
#         self.cls_weight = nn.Parameter(torch.randn(num_sets, output_dim)/output_dim)

        init_val = torch.randn(num_sets, output_dim)*0.01
        for ns in range(num_sets):
            init_val[ns, ns%output_dim] = 0.1
        self.cls_weight = nn.Parameter(init_val)
        
        self.cls_confidence = None
        
        
    def forward(self, x, hard=False):
#         self.cls_weight.data = torch.abs(self.cls_weight.data/self.cls_weight.data.sum(dim=1, keepdim=True))
        
        x = self.linear(x)
        if hard:
            x = torch.softmax(x*1e5, dim=1)
        else:
            x = torch.softmax(x*self.inv_temp, dim=1)
        self.cls_confidence = x
        c = torch.softmax(self.cls_weight, dim=1)
#         c = self.cls_weight
        return x@c ## since both are normalized, it is also normalized

In [17]:
class ConnectedClassifier_SoftKMeans(nn.Module):
    
    def __init__(self,input_dim, num_sets, output_dim, inv_temp=1):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.num_sets = num_sets
        self.inv_temp = nn.Parameter(torch.ones(1)*inv_temp)
        
        self.centers = nn.Parameter(torch.rand(num_sets, input_dim)*2-1)
        
#         self.cls_weight = nn.Parameter(torch.ones(num_sets, output_dim)/output_dim)

        init_val = torch.randn(num_sets, output_dim)*0.01
        for ns in range(num_sets):
            init_val[ns, ns%output_dim] = 0.1
        self.cls_weight = nn.Parameter(init_val)

        self.cls_confidence = None
        
        
    def forward(self, x, hard=False):
#         self.cls_weight.data = torch.abs(self.cls_weight.data/self.cls_weight.data.sum(dim=1, keepdim=True))
        
        x = x[:, :self.input_dim]
        dists = torch.cdist(x, self.centers)
        dists = dists/np.sqrt(self.input_dim) ### correction to make diagonal of unit square 1 in nD space
        
        if hard:
            x = torch.softmax(-dists*1e5, dim=1)
        else:
            x = torch.softmax(-dists*self.inv_temp, dim=1)
        self.cls_confidence = x
        c = torch.softmax(self.cls_weight, dim=1)
#         c = self.cls_weight
        return x@c ## since both are normalized, it is also normalized
#         return torch.softmax(x@self.cls_weight, dim=1)

In [18]:
# actf = irf.Swish
# flows = [
#     ActNorm(784),
#     irf.ResidualFlow(784, [784], activation=actf),
#     ActNorm(784),
#     irf.ResidualFlow(784, [784], activation=actf),
#     ActNorm(784),
#         ]

# model = SequentialFlow(flows)

In [19]:
actf = irf.Swish
flows = [
    ActNorm2D(1),
    irf.ConvResidualFlow(1, [16], activation=actf),
    irf.InvertiblePooling(2),
    ActNorm2D(4),
    irf.ConvResidualFlow(4, [64], activation=actf),
    irf.InvertiblePooling(2),
    ActNorm2D(16),
    irf.ConvResidualFlow(16, [64, 64], activation=actf),
    irf.Flatten(img_size=(16, 7, 7))
        ]

model = SequentialFlow(flows)

In [21]:
def get_children(module):
    child = list(module.children())
    if len(child) == 0:
        return [module]
    children = []
    for ch in child:
        grand_ch = get_children(ch)
        children+=grand_ch
    return children

In [33]:
def remove_spectral_norm(model):
    for child in get_children(model):
        if hasattr(child, 'weight'):
            print("Yes", child)
            try:
                nn.utils.remove_spectral_norm(child)
                print("Success")
            except:
                print("Failed")

In [34]:
remove_spectral_norm(model)

Yes Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Failed
Yes Conv2d(16, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Failed
Yes Conv2d(4, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Failed
Yes Conv2d(64, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Failed
Yes Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Failed
Yes Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Failed
Yes Conv2d(64, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Failed


In [15]:
# model = nn.Sequential(nn.Linear(784, 784, bias=False),
#                       nn.BatchNorm1d(784),
#                       nn.SELU(),
#                       nn.Linear(784, 784, bias=False),
#                       nn.BatchNorm1d(784),
#                       nn.SELU(),
#                      )

In [16]:
# model(torch.randn(3, 1, 28, 28)).shape

In [35]:
model.to(device)

SequentialFlow(
  (flows): ModuleList(
    (0): ActNorm2D()
    (1): ConvResidualFlow(
      (resblock): ModuleList(
        (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): Swish()
        (2): Conv2d(16, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (2): InvertiblePooling()
    (3): ActNorm2D()
    (4): ConvResidualFlow(
      (resblock): ModuleList(
        (0): Conv2d(4, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): Swish()
        (2): Conv2d(64, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (5): InvertiblePooling()
    (6): ActNorm2D()
    (7): ConvResidualFlow(
      (resblock): ModuleList(
        (0): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): Swish()
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): Swish()
        (4): Conv2d(64, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )


In [36]:
classifier = ConnectedClassifier_SoftKMeans(784, 100, 10)
# classifier = ConnectedClassifier_Softmax(784, 100, 10)
classifier = classifier.to(device)

## Model Train

In [37]:
learning_rate = 0.0003
batch_size = 50

In [38]:
train_loader = data.DataLoader(dataset=train_dataset, num_workers=4, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(dataset=test_dataset, num_workers=4, batch_size=batch_size, shuffle=False)

In [39]:
# criterion = nn.NLLLoss()
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(list(model.parameters())+list(classifier.parameters()), lr=learning_rate)
# optimizer = optim.SGD(model.parameters(), lr=0.1)

print("number of params: ", sum(p.numel() for p in model.parameters()))

number of params:  60467


In [40]:
# for p in model.parameters():
#     print(torch.isnan(p).type(torch.float32).sum())

In [41]:
# model(torch.randn(10, 784).to(device)).shape

In [42]:
xx = iter(test_loader).next()[0]
xx.shape

torch.Size([50, 1, 28, 28])

In [43]:
losses = []
train_accs = []
test_accs = []
EPOCHS = 25#50

In [44]:
index = 0
for epoch in range(EPOCHS):
    train_acc = 0
    train_count = 0
    for xx, yy in tqdm(train_loader):
        xx, yy = xx.to(device), yy.to(device)
#     for xx, yy in tqdm(test_loader):

        yout = model(xx)
#         print(yout)
        yout = classifier(yout)    
#         print(yout)
        loss = criterion(yout, yy)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(float(loss))

        outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
        correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
        train_acc += correct
        train_count += len(outputs)
#         break

    train_accs.append(float(train_acc)/train_count*100)
    train_acc = 0
    train_count = 0

    print(f'Epoch: {epoch}:{index},  Loss:{float(loss)}')
    test_count = 0
    test_acc = 0
    for xx, yy in tqdm(test_loader):
        xx, yy = xx.to(device), yy.to(device)
        with torch.no_grad():
            yout = classifier(model(xx))    
        outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
        correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
        test_acc += correct
        test_count += len(xx)
    test_accs.append(float(test_acc)/test_count*100)
    print(f'Train Acc:{train_accs[-1]:.2f}%, Test Acc:{test_accs[-1]:.2f}%')
    print()

### after each class index is finished training
print(f'\t-> Train Acc {max(train_accs)} ; Test Acc {max(test_accs)}')

  0%|          | 0/200 [00:00<?, ?it/s]

Epoch: 0:0,  Loss:2.2961928844451904


  0%|          | 0/1200 [00:00<?, ?it/s]

Train Acc:29.25%, Test Acc:40.92%



  9%|▉         | 107/1200 [00:06<01:07, 16.23it/s]


KeyboardInterrupt: 

In [33]:
classifier.inv_temp

Parameter containing:
tensor([17.4657], device='cuda:0', requires_grad=True)

### Hard test accuracy with count per classifier

In [34]:
test_count = 0
test_acc = 0
set_count = torch.zeros(classifier.num_sets).to(device)
for xx, yy in tqdm(test_loader):
    xx, yy = xx.to(device), yy.to(device)
    with torch.no_grad():
        yout = classifier(model(xx), hard=True)
        set_indx, count = torch.unique(torch.argmax(classifier.cls_confidence, dim=1), return_counts=True) 
        set_count[set_indx] += count
    outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
    correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
    test_acc += correct
    test_count += len(xx)

print(f'Hard Test Acc:{float(test_acc)/test_count*100:.2f}%')
print(set_count.type(torch.long).tolist())

100%|██████████| 200/200 [00:00<00:00, 211.18it/s]

Hard Test Acc:91.64%
[0, 19, 69, 0, 0, 0, 930, 0, 0, 0, 0, 965, 0, 10, 0, 0, 0, 0, 0, 0, 0, 0, 903, 0, 0, 0, 0, 1034, 22, 0, 3, 0, 6, 0, 0, 0, 5, 0, 0, 941, 0, 0, 22, 0, 700, 1, 0, 0, 0, 0, 0, 0, 0, 0, 236, 0, 0, 0, 0, 0, 12, 0, 0, 92, 31, 2, 0, 0, 0, 0, 949, 0, 0, 0, 0, 1007, 0, 0, 0, 0, 42, 0, 0, 905, 0, 0, 78, 0, 945, 23, 0, 0, 0, 0, 0, 0, 9, 0, 39, 0]


### Hard train accuracy with count per classifier

In [35]:
test_count = 0
test_acc = 0
set_count = torch.zeros(classifier.num_sets).to(device)
for xx, yy in tqdm(train_loader):
    xx, yy = xx.to(device), yy.to(device)
    with torch.no_grad():
        yout = classifier(model(xx), hard=True)
        set_indx, count = torch.unique(torch.argmax(classifier.cls_confidence, dim=1), return_counts=True) 
        set_count[set_indx] += count
    outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
    correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
    test_acc += correct
    test_count += len(xx)

print(f'Hard Train Acc:{float(test_acc)/test_count*100:.2f}%')
print(set_count.type(torch.long).tolist())

100%|██████████| 1200/1200 [00:04<00:00, 246.07it/s]

Hard Train Acc:94.52%
[0, 127, 413, 0, 0, 0, 5391, 0, 0, 0, 0, 5835, 0, 68, 0, 0, 0, 0, 0, 0, 7, 0, 5506, 0, 0, 1, 1, 6138, 126, 0, 7, 0, 11, 0, 0, 0, 41, 0, 0, 5715, 3, 0, 147, 0, 4089, 2, 0, 0, 0, 1, 0, 3, 0, 7, 1610, 0, 0, 0, 0, 0, 83, 0, 0, 543, 227, 6, 0, 0, 4, 0, 5752, 0, 0, 10, 0, 5986, 0, 0, 2, 0, 278, 0, 0, 5409, 0, 0, 397, 0, 5731, 156, 0, 0, 0, 0, 0, 0, 39, 0, 128, 0]


In [36]:
#### Classifiers that enclose any data
torch.count_nonzero(set_count)

tensor(39, device='cuda:0')

In [37]:
#### classifier with class representation
torch.argmax(classifier.cls_weight, dim=1)

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 6, 3, 6, 5, 6, 5, 8, 9, 0, 1, 2, 3,
        4, 5, 6, 7, 8, 9, 0, 3, 2, 3, 6, 5, 6, 5, 6, 9, 0, 1, 2, 0, 4, 5, 6, 5,
        8, 9, 6, 1, 2, 3, 4, 5, 6, 5, 8, 9, 0, 1, 6, 3, 4, 5, 6, 7, 8, 9, 0, 1,
        6, 3, 4, 5, 6, 7, 8, 9, 0, 1, 6, 3, 4, 8, 6, 5, 8, 9, 0, 3, 6, 6, 6, 5,
        6, 5, 8, 9], device='cuda:0')

In [38]:
# The class labels are same as that of initialized
# tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3,
#         4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7,
#         8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1,
#         2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5,
#         6, 7, 8, 9], device='cuda:0')

In [39]:
classifier.cls_weight

Parameter containing:
tensor([[ 4.5707e+00, -6.1187e+00, -5.9843e+00, -5.7596e+00, -6.2395e+00,
         -6.7145e+00, -1.3000e+00, -6.7182e+00, -6.4216e+00, -6.7123e+00],
        [-4.5537e+00,  5.8321e+00, -5.2303e+00, -3.1670e+00, -5.1807e+00,
         -5.5590e+00, -4.8234e+00, -5.5957e+00, -5.5326e+00, -5.5693e+00],
        [-7.3217e+00, -7.5306e+00,  1.0043e+01, -7.4558e+00, -6.1344e+00,
         -7.6038e+00, -6.0353e+00, -7.6112e+00, -7.5306e+00, -7.6020e+00],
        [ 1.0200e+00, -1.3880e+00, -2.0284e+00,  1.8134e+00, -2.2929e+00,
         -2.9024e+00,  7.4698e-01, -3.0175e+00, -2.5613e+00, -2.9651e+00],
        [-5.7246e+00, -5.9981e+00, -4.0205e+00, -5.7197e+00,  7.6831e+00,
         -6.1818e+00, -3.5888e+00, -6.1914e+00, -6.0481e+00, -6.1877e+00],
        [-1.2317e+00, -1.5968e+00, -1.6302e+00, -1.3119e+00, -1.7657e+00,
          2.0491e+00, -5.8906e-01, -7.8146e-01, -2.0993e-01,  2.7617e-01],
        [-8.5499e+00, -8.8448e+00, -6.4159e+00, -8.7495e+00, -6.4786e+00,
         -

In [40]:
# torch.unique(torch.argmax(classifier.cls_confidence, dim=1), return_counts=True)

In [41]:
classifier.inv_temp

Parameter containing:
tensor([17.4657], device='cuda:0', requires_grad=True)

In [42]:
### example output per classifier
yout[5]

tensor([1.5721e-08, 2.1479e-08, 8.7105e-09, 1.0000e+00, 9.6352e-09, 7.7206e-09,
        1.0086e-08, 7.6261e-09, 7.7955e-09, 7.6698e-09], device='cuda:0')

In [43]:
asdfsdf ## to break the code

NameError: name 'asdfsdf' is not defined

### analyze per classifier accuracy

In [44]:
test_count = 0
test_acc = 0
set_count = torch.zeros(classifier.num_sets).to(device)
set_acc = torch.zeros(classifier.num_sets).to(device)
for xx, yy in tqdm(train_loader):
    xx, yy = xx.to(device), yy.to(device)
    with torch.no_grad():
        yout = classifier(model(xx), hard=True)
        
    cls_indx = torch.argmax(classifier.cls_confidence, dim=1)
    set_indx, count = torch.unique(cls_indx, return_counts=True) 
    set_count[set_indx] += count
    
    outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
    correct = (outputs == yy.data.cpu().numpy()).astype(float)
    
    ### class_index has 100 possible values
    for i, c in enumerate(correct):
        set_acc[cls_indx[i]] += c
    
#     print(set_acc.sum(), set_count.sum())
#     break
    test_acc += correct.sum()
    test_count += len(xx)

print(f'Hard Train Acc:{float(test_acc)/test_count*100:.2f}%')
print(set_count.type(torch.long).tolist())

100%|██████████| 1200/1200 [00:06<00:00, 176.71it/s]

Hard Train Acc:94.52%
[0, 127, 413, 0, 0, 0, 5391, 0, 0, 0, 0, 5835, 0, 68, 0, 0, 0, 0, 0, 0, 7, 0, 5506, 0, 0, 1, 1, 6138, 126, 0, 7, 0, 11, 0, 0, 0, 41, 0, 0, 5715, 3, 0, 147, 0, 4089, 2, 0, 0, 0, 1, 0, 3, 0, 7, 1610, 0, 0, 0, 0, 0, 83, 0, 0, 543, 227, 6, 0, 0, 4, 0, 5752, 0, 0, 10, 0, 5986, 0, 0, 2, 0, 278, 0, 0, 5409, 0, 0, 397, 0, 5731, 156, 0, 0, 0, 0, 0, 0, 39, 0, 128, 0]


In [45]:
set_acc/set_count

tensor([   nan, 0.9370, 0.8741,    nan,    nan,    nan, 0.8611,    nan,    nan,
           nan,    nan, 0.9962,    nan, 0.8382,    nan,    nan,    nan,    nan,
           nan,    nan, 1.0000,    nan, 0.9130,    nan,    nan, 1.0000, 1.0000,
        0.9653, 0.9444,    nan, 1.0000,    nan, 0.7273,    nan,    nan,    nan,
        0.9024,    nan,    nan, 0.9890, 1.0000,    nan, 0.8367,    nan, 0.9359,
        1.0000,    nan,    nan,    nan, 1.0000,    nan, 1.0000,    nan, 0.8571,
        0.8522,    nan,    nan,    nan,    nan,    nan, 0.8072,    nan,    nan,
        0.9503, 0.9163, 1.0000,    nan,    nan, 1.0000,    nan, 0.9080,    nan,
           nan, 0.9000,    nan, 0.9932,    nan,    nan, 0.5000,    nan, 0.7914,
           nan,    nan, 0.9440,    nan,    nan, 0.7456,    nan, 0.9949, 0.9936,
           nan,    nan,    nan,    nan,    nan,    nan, 0.7436,    nan, 0.8750,
           nan], device='cuda:0')

In [46]:
for i, (cnt, acc, cls) in enumerate(zip(set_count.type(torch.long).tolist(),
                                   (set_acc/set_count).tolist(),
                                   torch.argmax(classifier.cls_weight, dim=1).tolist())):
    if cnt == 0: continue
    print(f"{i},\t {cnt},\t {cls}\t {acc*100:.2f}%")

1,	 127,	 1	 93.70%
2,	 413,	 2	 87.41%
6,	 5391,	 6	 86.11%
11,	 5835,	 1	 99.62%
13,	 68,	 3	 83.82%
20,	 7,	 0	 100.00%
22,	 5506,	 2	 91.30%
25,	 1,	 5	 100.00%
26,	 1,	 6	 100.00%
27,	 6138,	 7	 96.53%
28,	 126,	 8	 94.44%
30,	 7,	 0	 100.00%
32,	 11,	 2	 72.73%
36,	 41,	 6	 90.24%
39,	 5715,	 9	 98.90%
40,	 3,	 0	 100.00%
42,	 147,	 2	 83.67%
44,	 4089,	 4	 93.59%
45,	 2,	 5	 100.00%
49,	 1,	 9	 100.00%
51,	 3,	 1	 100.00%
53,	 7,	 3	 85.71%
54,	 1610,	 4	 85.22%
60,	 83,	 0	 80.72%
63,	 543,	 3	 95.03%
64,	 227,	 4	 91.63%
65,	 6,	 5	 100.00%
68,	 4,	 8	 100.00%
70,	 5752,	 0	 90.80%
73,	 10,	 3	 90.00%
75,	 5986,	 5	 99.32%
78,	 2,	 8	 50.00%
80,	 278,	 0	 79.14%
83,	 5409,	 3	 94.40%
86,	 397,	 6	 74.56%
88,	 5731,	 8	 99.49%
89,	 156,	 9	 99.36%
96,	 39,	 6	 74.36%
98,	 128,	 8	 87.50%


### Benchmark - Robustness

In [50]:
import foolbox as fb
import foolbox.attacks as fa

In [59]:
epsilons = [0.001, 0.01, 0.03, 0.1, 0.3, 0.5, 1.0]
# epsilons = [0.0005, 0.001, 0.0015, 0.002, 0.003, 0.005, 0.01, 0.02, 0.03, 0.1, 0.3, 0.5, 1.0,]

#### Benchmark on full dataset

In [60]:
attacks = [
    fa.FGSM(),
    fa.LinfPGD(),
    fa.LinfBasicIterativeAttack(),
#     fa.LinfAdditiveUniformNoiseAttack(),
#     fa.LinfDeepFoolAttack(),
]
atk_names = [
    "FGSM", 
    "LinfPGD", 
    "LinfBasicIterativeAttack", 
#     "LinfAdditiveUniformNoiseAttack", 
#     "LinfDeepFoolAttack",
]

In [61]:
def get_attack_success(model, attack, dataloader):
    fmodel = fb.PyTorchModel(model.eval(), bounds=(-1, 1))   
    success_per_eps = []

    for eps in epsilons:
        count = 0
        total = 0
        print(f"Running one epoch attack for eps: {eps}")
        for i, (images, labels) in enumerate(tqdm(dataloader)):
            images, labels = images.to(device), labels.to(device)
            _, _, success = attack(fmodel, images, labels, epsilons=[eps])
            count += int(torch.count_nonzero(success))
            total += torch.numel(success)
#             break
        success_per_eps.append(count/total)
    return success_per_eps

In [62]:
# get_attack_success(model, attacks[0], test_loader)

### Compute the robustness

In [63]:
outputs = {}

model.eval()
### load model and compute attack_success
for j in range(len(attacks)):
    attack = attacks[j]
    atname = atk_names[j]

    print(f"Attacking on model using {atname}")
    succ_eps = get_attack_success(model, attack, test_loader)
    outputs[atname] = succ_eps
    print(f"Success rate is : {succ_eps}")

  0%|          | 0/200 [00:00<?, ?it/s]

Attacking on model using FGSM
Running one epoch attack for eps: 0.001


  0%|          | 0/200 [00:00<?, ?it/s]

Running one epoch attack for eps: 0.01


  0%|          | 0/200 [00:00<?, ?it/s]

...

Running one epoch attack for eps: 0.5


  0%|          | 0/200 [00:00<?, ?it/s]

Running one epoch attack for eps: 1.0


100%|██████████| 200/200 [00:12<00:00, 16.20it/s]

Success rate is : [0.9988, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
